In [1]:
import os
import sys

import numpy as np
import torch

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from core.integrate import PanopticFusionScalableTSDFVolume


In [2]:
device = "cuda:0"

tsdf_volume = PanopticFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=device,
)

panoptic_tsdf_volume = PanopticFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=device,
)


In [3]:
scene_dir = "/scratch/quanta/Experiments/feature-instance-fusion-02/scannet_scene0000_00/"

save_dir = scene_dir + "tsdf_label_extend_grounded_sam/"
label_pth = save_dir + "labels.pt"

tsdf_volume.load(scene_dir + 'tsdf/tsdf_volume_unpruned.pt')
tsdf_volume.load_instance(label_pth)

panoptic_label_pth = scene_dir + 'tsdf_panoptic_grounded_sam/panoptic_labels.pt'
panoptic_tsdf_volume.load(scene_dir + 'tsdf/tsdf_volume_unpruned.pt')
panoptic_tsdf_volume.load_instance(panoptic_label_pth)


In [4]:
scan_dir = "/scratch/quanta/Datasets/ScanNet/scans/scene0000_00/"

low_res_ply_pth = scan_dir + "scene0000_00_vh_clean_2.ply"
low_res_segs_pth = scan_dir + "scene0000_00_vh_clean_2.0.010000.segs.json"
low_res_instance_pth = scan_dir + "scene0000_00.aggregation.json"

high_res_ply_pth = scan_dir + "scene0000_00_vh_clean.ply"
high_res_segs_pth = scan_dir + "scene0000_00_vh_clean.segs.json"
high_res_instance_pth = scan_dir + "scene0000_00_vh_clean.aggregation.json"


In [5]:
from plyfile import PlyData


In [6]:
low_res_ply = PlyData.read(low_res_ply_pth)
low_res_verts = np.vstack(
            [
                low_res_ply["vertex"].data["x"],
                low_res_ply["vertex"].data["y"],
                low_res_ply["vertex"].data["z"],
            ]
        ).transpose()
low_res_faces = np.vstack(low_res_ply["face"].data["vertex_indices"])


In [7]:
high_res_ply = PlyData.read(high_res_ply_pth)
high_res_verts = np.vstack(
            [
                high_res_ply["vertex"].data["x"],
                high_res_ply["vertex"].data["y"],
                high_res_ply["vertex"].data["z"],
            ]
        ).transpose()
high_res_faces = np.vstack(high_res_ply["face"].data["vertex_indices"])


In [8]:
color_set = np.random.random(size=(high_res_verts.shape[0] + 2, 3))
color_set[0, :] = 0.0


In [9]:
import open3d as o3d

low_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(low_res_verts),
    triangles=o3d.utility.Vector3iVector(low_res_faces),
)
low_res_mesh_labels = tsdf_volume.extract_label_on_grid(verts=low_res_verts, device=device)[0]
low_res_color = color_set[low_res_mesh_labels]
low_res_mesh.vertex_colors = o3d.utility.Vector3dVector(low_res_color)
# o3d.io.write_triangle_mesh(filename="low_res_instance_color.ply", mesh=low_res_mesh)

# np.save('low_res_mesh_labels.npy', low_res_mesh_labels)


In [10]:
high_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(high_res_verts),
    triangles=o3d.utility.Vector3iVector(high_res_faces),
)
high_res_mesh_labels = tsdf_volume.extract_label_on_grid(verts=high_res_verts, device=device)[0]
high_res_color = color_set[high_res_mesh_labels]
high_res_mesh.vertex_colors = o3d.utility.Vector3dVector(high_res_color)
# o3d.io.write_triangle_mesh(filename="high_res_instance_color.ply", mesh=high_res_mesh)

# np.save('high_res_mesh_labels.npy', high_res_mesh_labels)


In [11]:
panoptic_low_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(low_res_verts),
    triangles=o3d.utility.Vector3iVector(low_res_faces),
)
panoptic_low_res_mesh_labels = panoptic_tsdf_volume.extract_label_on_grid(verts=low_res_verts, device=device)[0]
panoptic_low_res_color = color_set[panoptic_low_res_mesh_labels]
panoptic_low_res_mesh.vertex_colors = o3d.utility.Vector3dVector(panoptic_low_res_color)
# o3d.io.write_triangle_mesh(filename="panoptic_low_res_instance_color.ply", mesh=panoptic_low_res_mesh)

# np.save('panoptic_low_res_mesh_labels.npy', panoptic_low_res_mesh_labels)


In [12]:
panoptic_high_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(high_res_verts),
    triangles=o3d.utility.Vector3iVector(high_res_faces),
)
panoptic_high_res_mesh_labels = panoptic_tsdf_volume.extract_label_on_grid(verts=high_res_verts, device=device)[0]
panoptic_high_res_color = color_set[panoptic_high_res_mesh_labels]
panoptic_high_res_mesh.vertex_colors = o3d.utility.Vector3dVector(panoptic_high_res_color)
# o3d.io.write_triangle_mesh(filename="panoptic_high_res_instance_color.ply", mesh=panoptic_high_res_mesh)

# np.save('panoptic_high_res_mesh_labels.npy', panoptic_high_res_mesh_labels)


# load ground truth instance segmentation

In [13]:
import json
with open(low_res_segs_pth) as f:
    low_res_segs = np.array(json.load(f)['segIndices'])

low_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(low_res_verts),
    triangles=o3d.utility.Vector3iVector(low_res_faces),
)
low_res_mesh_labels = tsdf_volume.extract_label_on_grid(verts=low_res_verts, device=device)[0]
low_res_color = color_set[low_res_segs]
low_res_mesh.vertex_colors = o3d.utility.Vector3dVector(low_res_color)
# o3d.io.write_triangle_mesh(filename="low_res_over_seg_color.ply", mesh=low_res_mesh)


In [14]:
low_res_instance_id = np.zeros(shape=(low_res_verts.shape[0],), dtype=int)
low_res_instance_id_to_label = {}
with open(low_res_instance_pth) as f:
    a = json.load(f)
    for segs in a["segGroups"]:
        low_res_instance_id[segs["segments"]] = segs["objectId"] + 1
        low_res_instance_id_to_label[segs["objectId"] + 1] = segs["label"]

low_res_gt_instance_id = low_res_instance_id[low_res_segs]

low_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(low_res_verts),
    triangles=o3d.utility.Vector3iVector(low_res_faces),
)
low_res_mesh_labels = tsdf_volume.extract_label_on_grid(verts=low_res_verts, device=device)[0]
low_res_color = color_set[low_res_gt_instance_id]
low_res_mesh.vertex_colors = o3d.utility.Vector3dVector(low_res_color)
# o3d.io.write_triangle_mesh(filename="low_res_instance_seg_color.ply", mesh=low_res_mesh)


In [15]:
import json
with open(high_res_segs_pth) as f:
    high_res_segs = np.array(json.load(f)['segIndices'])

high_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(high_res_verts),
    triangles=o3d.utility.Vector3iVector(high_res_faces),
)
high_res_color = color_set[high_res_segs]
high_res_mesh.vertex_colors = o3d.utility.Vector3dVector(high_res_color)
# o3d.io.write_triangle_mesh(filename="high_res_over_seg_color.ply", mesh=high_res_mesh)


In [16]:
high_res_instance_id = np.zeros(shape=(high_res_verts.shape[0],), dtype=int)
high_res_instance_id_to_label = {}
with open(low_res_instance_pth) as f:
    a = json.load(f)
    for segs in a["segGroups"]:
        high_res_instance_id[segs["segments"]] = segs["objectId"] + 1
        high_res_instance_id_to_label[segs["objectId"] + 1] = segs["label"]

high_res_gt_instance_id = high_res_instance_id[high_res_segs]

high_res_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(high_res_verts),
    triangles=o3d.utility.Vector3iVector(high_res_faces),
)
high_res_color = color_set[high_res_gt_instance_id]
high_res_mesh.vertex_colors = o3d.utility.Vector3dVector(high_res_color)
# o3d.io.write_triangle_mesh(filename="high_res_instance_seg_color.ply", mesh=high_res_mesh)


## number of objectes for two methods and ground truth

In [17]:
np.unique(high_res_gt_instance_id).shape[0], np.unique(high_res_mesh_labels).shape[0], np.unique(panoptic_high_res_mesh_labels).shape[0]


(68, 424, 837)

### Naviely compute IoU

later, we should merge instance with high IoPred to the instance, since there is a many to one correspondence


In [18]:
gt_instance = torch.from_numpy(high_res_gt_instance_id).unique(sorted=True, return_inverse=True)[1].numpy()
graph_connect_instance = torch.from_numpy(high_res_mesh_labels).unique(sorted=True, return_inverse=True)[1].numpy()
panoptic_instance = torch.from_numpy(panoptic_high_res_mesh_labels).unique(sorted=True, return_inverse=True)[1].numpy()


In [19]:
device = 'cuda:0'
torch.cuda.empty_cache()
gt_one_hot = torch.nn.functional.one_hot(torch.from_numpy(gt_instance).to(device))
graph_one_hot = torch.nn.functional.one_hot(torch.from_numpy(graph_connect_instance).to(device))
panoptic_one_hot = torch.nn.functional.one_hot(torch.from_numpy(panoptic_instance).to(device))


In [22]:
from tqdm import tqdm

graph_I = torch.cat([(graph_one_hot * mask.reshape(-1, 1)).count_nonzero(dim=0).reshape(1, -1) for mask in tqdm(gt_one_hot.movedim(0, -1))], dim=0).cpu()
graph_U = torch.cat([(graph_one_hot + mask.reshape(-1, 1)).count_nonzero(dim=0).reshape(1, -1) for mask in tqdm(gt_one_hot.movedim(0, -1))], dim=0).cpu()

torch.cuda.empty_cache()

panoptic_I = torch.cat([(panoptic_one_hot * mask.reshape(-1, 1)).count_nonzero(dim=0).reshape(1, -1) for mask in tqdm(gt_one_hot.movedim(0, -1))], dim=0).cpu()
panoptic_U = torch.cat([(panoptic_one_hot + mask.reshape(-1, 1)).count_nonzero(dim=0).reshape(1, -1) for mask in tqdm(gt_one_hot.movedim(0, -1))], dim=0).cpu()

torch.cuda.empty_cache()


100%|██████████| 68/68 [00:04<00:00, 15.61it/s] 


In [38]:
graph_IoU = graph_I / graph_U
panoptic_IoU = panoptic_I / panoptic_U

graph_IoS = graph_I / graph_one_hot.count_nonzero(dim=0).reshape(1, -1).cpu()
panoptic_IoS = panoptic_I / panoptic_one_hot.count_nonzero(dim=0).reshape(1, -1).cpu()


In [29]:
torch.save(graph_IoU, 'graph_IoU.pt')
torch.save(panoptic_IoU, 'panoptic_IoU.pt')


In [30]:
graph_max, graph_argmax = graph_IoU.max(dim=1)
panoptic_max, panoptic_argmax = panoptic_IoU.max(dim=1)


In [34]:
(graph_max > panoptic_max).float().mean()


tensor(0.5588)

In [35]:
graph_max.mean(), panoptic_max.mean()


(tensor(0.4602), tensor(0.4809))

In [43]:
graph_IoS.max(), panoptic_IoS.max()


(tensor(1.), tensor(1.))